**Задача 1: предсказание уровня удовлетворённости сотрудника**

`id` — уникальный идентификатор сотрудника;<br>
`dept` — отдел, в котором работает сотрудник;<br>
`level` — уровень занимаемой должности;<br>
`workload` — уровень загруженности сотрудника;<br>
`employment_years` — длительность работы в компании (в годах);<br>
`last_year_promo` — показывает, было ли повышение за последний год;<br>
`last_year_violations` — показывает, нарушал ли сотрудник трудовой договор за последний год;<br>
`supervisor_evaluation` — оценка качества работы сотрудника, которую дал руководитель;<br>
`salary` — ежемесячная зарплата сотрудника;<br>
`job_satisfaction_rate` — уровень удовлетворённости сотрудника работой в компании, целевой признак.<br>

1. Загрузка данных;
2. Предобработка данных;
3. Исследовательский анализ данных;
4. Подготовка данных:
    Подготовку признаков выполнить в пайплайне, дополнив пайплайн шагом предобработки. При кодировании учитывайть особенности признаков и моделей и использовать как минимум два кодировщика;
5. Обучение моделей: 
    Обучить как минимум две модели: линейную и дерево решений. Подобрать гиперпараметры как минимум для одной модели. Выбрать лучшую модель и проверить её качество. Выбор делайть на основе метрики — SMAPE (англ. symmetric mean absolute percentage error, «симметричное среднее абсолютное процентное отклонение»). Написать функцию, которая принимает на вход массивы NumPy или объекты Series в pandas и возвращает значение метрики SMAPE. Использовать эту метрику при подборе гиперпараметров и оценке качества моделей. Критерий успеха: SMAPE ≤ 15 на тестовой выборке. В решении сохранить работу со всеми моделями, которые пробовали. Сделать выводы.
6. Выводы.


**Задача 2: предсказание увольнения сотрудника из компании**

1. Загрузка данных;
2. Предобработка данных;
3. Исследовательский анализ данных;
    1. Провести исследовательский анализ данных;
    2. Составить портрет «уволившегося сотрудника». Например, можно узнать, в каком отделе с большей вероятностью работает уволившийся сотрудник и какой у него уровень загруженности. Также можно сравнить среднее значение зарплаты ушедших сотрудников с теми, кто остался в компании;
    3. Аналитики утверждают, что уровень удовлетворённости сотрудника работой в компании влияет на то, уволится ли сотрудник. Проверить это утверждение: визуализируйте и сравните распределения признака `job_satisfaction_rate` для ушедших и оставшихся сотрудников. Используйте данные с обоими целевыми признаками тестовой выборки.
4. Добавление нового входного признака;
    Допустим, `job_satisfaction_rate` и `quit` действительно связаны и получено необходимое значение метрики в первой задаче. Тогда добавить `job_satisfaction_rate`, предсказанный лучшей моделью первой задачи, к входным признакам второй задачи.
5. Подготовка данных;
    Подготовьте признаки так же, как и в первой задаче.
6. Обучение модели;
    Обучите как минимум три модели. Как минимум для двух из них подберите гиперпараметры. Проверьте качество лучшей модели. Метрика оценки качества в этой задаче — *ROC-AUC*. Критерий успеха: *ROC-AUC* ≥ 0.91 на тестовой выборке. Напомним: отбор признаков часто помогает улучшить метрику.
7. Выводы.

**Общий вывод в конце**

In [3]:
!pip install --upgrade scikit-learn
!pip install --upgrade shap matplotlib
!pip install shap
!pip install phik
!pip install --upgrade seaborn

     |████████████████████████████████| 13.5 MB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 307 kB 88.1 MB/s eta 0:00:01
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1
     |████████████████████████████████| 994 kB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 8.3 MB 56.9 MB/s eta 0:00:01
     |████████████████████████████████| 4.7 MB 81.0 MB/s eta 0:00:01
     |████████████████████████████████| 321 kB 71.4 MB/s eta 0:00:01
     |████████████████████████████████| 8.3 MB 85.5 MB/s eta 0:00:01
     |████████████████████████████████| 8.3 MB 76.9 MB/s eta 0:00:01
     |████████████████████████████████| 8.3 MB 45.8 MB/s eta 0:00:01
     |████████████████████████████████| 8.3 MB 49

In [4]:
from scipy import stats

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import shap
import phik

from sklearn.compose import ColumnTransformer
from sklearn.metrics import (
    f1_score, 
    recall_score, 
    roc_auc_score, 
    make_scorer
)
from sklearn.model_selection import (
    RandomizedSearchCV,
    train_test_split,
    KFold,
    StratifiedKFold
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    MinMaxScaler,
    OneHotEncoder,
    OrdinalEncoder,
    RobustScaler,
    StandardScaler,
    LabelEncoder
)
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor, DummyClassifier

## Предсказание уровня удовлетворённости сотрудника

### Загрузка данных

In [5]:
train_df=pd.read_csv('/datasets/train_job_satisfaction_rate.csv', index_col='id')
testf_df=pd.read_csv('/datasets/test_features.csv', index_col='id')
testt_df=pd.read_csv('/datasets/test_target_job_satisfaction_rate.csv', index_col='id')

In [6]:
train_df.head(10)

,dept,level,workload,employment_years,last_year_promo,last_year_violations,supervisor_evaluation,salary,job_satisfaction_rate
id,,,,,,,,,
155278,sales,junior,medium,2,no,no,1,24000,0.58
653870,hr,junior,high,2,no,no,5,38400,0.76
184592,sales,junior,low,1,no,no,2,12000,0.11
171431,technology,junior,low,4,no,no,2,18000,0.37
693419,hr,junior,medium,1,no,no,3,22800,0.20
405448,hr,middle,low,7,no,no,4,30000,0.78
857135,sales,sinior,medium,9,no,no,3,56400,0.56
400657,purchasing,middle,high,9,no,no,3,52800,0.44
198846,hr,junior,low,1,no,no,2,13200,0.14


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 155278 to 338347
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dept                   3994 non-null   object 
 1   level                  3996 non-null   object 
 2   workload               4000 non-null   object 
 3   employment_years       4000 non-null   int64  
 4   last_year_promo        4000 non-null   object 
 5   last_year_violations   4000 non-null   object 
 6   supervisor_evaluation  4000 non-null   int64  
 7   salary                 4000 non-null   int64  
 8   job_satisfaction_rate  4000 non-null   float64
dtypes: float64(1), int64(3), object(5)
memory usage: 312.5+ KB


In [8]:
testf_df.sample(10)

,dept,level,workload,employment_years,last_year_promo,last_year_violations,supervisor_evaluation,salary
id,,,,,,,,
196510,purchasing,middle,low,4,no,no,4,19200
524053,marketing,junior,medium,1,no,yes,4,27600
282510,sales,middle,medium,7,no,no,4,43200
356579,technology,middle,low,7,no,no,2,21600
222648,sales,junior,medium,1,no,no,4,27600
615683,marketing,sinior,high,8,no,no,3,79200
111132,purchasing,middle,low,3,no,no,3,19200
445276,technology,junior,low,1,no,no,3,22800
224309,sales,sinior,medium,8,no,no,3,58800


In [9]:
testf_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 485046 to 771859
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   dept                   1998 non-null   object
 1   level                  1999 non-null   object
 2   workload               2000 non-null   object
 3   employment_years       2000 non-null   int64 
 4   last_year_promo        2000 non-null   object
 5   last_year_violations   2000 non-null   object
 6   supervisor_evaluation  2000 non-null   int64 
 7   salary                 2000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 140.6+ KB


In [10]:
testt_df.sample(10)

,job_satisfaction_rate
id,
472068,0.88
955747,0.87
926986,0.57
529747,0.77
763138,0.50
923671,0.88
611939,0.32
349095,0.65
309991,0.67


In [11]:
testt_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 130604 to 648995
Data columns (total 1 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   job_satisfaction_rate  2000 non-null   float64
dtypes: float64(1)
memory usage: 31.2 KB


Есть незначительное количество пропусков. Типы данных соответствуют смыслу, названия столбцов записаны в принятом виде.

### Предобработка данных

Найдем уникальные значения:

In [12]:
def uniques(df):
    for col in df.select_dtypes(exclude='number').columns.tolist():
        unique_values = df[col].unique()
        print(f"Уникальные значения в столбце '{col}': {unique_values}")

In [13]:
uniques(train_df)

Уникальные значения в столбце 'dept': ['sales' 'hr' 'technology' 'purchasing' 'marketing' nan]
Уникальные значения в столбце 'level': ['junior' 'middle' 'sinior' nan]
Уникальные значения в столбце 'workload': ['medium' 'high' 'low']
Уникальные значения в столбце 'last_year_promo': ['no' 'yes']
Уникальные значения в столбце 'last_year_violations': ['no' 'yes']


In [14]:
uniques(testf_df)

Уникальные значения в столбце 'dept': ['marketing' 'hr' 'sales' 'purchasing' 'technology' nan ' ']
Уникальные значения в столбце 'level': ['junior' 'middle' 'sinior' nan]
Уникальные значения в столбце 'workload': ['medium' 'low' 'high' ' ']
Уникальные значения в столбце 'last_year_promo': ['no' 'yes']
Уникальные значения в столбце 'last_year_violations': ['no' 'yes']


Заменим пропуски вида " " на *NaN*

In [15]:
testf_df['dept'] = testf_df['dept'].replace({' ': np.nan})
testf_df['workload'] = testf_df['workload'].replace({' ': np.nan})

In [16]:
train_df.describe(include='all')

,dept,level,workload,employment_years,last_year_promo,last_year_violations,supervisor_evaluation,salary,job_satisfaction_rate
count,3994,3996,4000,4000.000000,4000,4000,4000.000000,4000.000000,4000.000000
unique,5,3,3,NaN,2,2,NaN,NaN,NaN
top,sales,junior,medium,NaN,no,no,NaN,NaN,NaN
freq,1512,1894,2066,NaN,3880,3441,NaN,NaN,NaN
mean,NaN,NaN,NaN,3.718500,NaN,NaN,3.476500,33926.700000,0.533995
std,NaN,NaN,NaN,2.542513,NaN,NaN,1.008812,14900.703838,0.225327
min,NaN,NaN,NaN,1.000000,NaN,NaN,1.000000,12000.000000,0.030000
25%,NaN,NaN,NaN,2.000000,NaN,NaN,3.000000,22800.000000,0.360000
50%,NaN,NaN,NaN,3.000000,NaN,NaN,4.000000,30000.000000,0.560000
75%,NaN,NaN,NaN,6.000000,NaN,NaN,4.000000,43200.000000,0.710000


In [17]:
testf_df.describe(include='all')

,dept,level,workload,employment_years,last_year_promo,last_year_violations,supervisor_evaluation,salary
count,1997,1999,1999,2000.000000,2000,2000,2000.000000,2000.000000
unique,5,3,3,NaN,2,2,NaN,NaN
top,sales,junior,medium,NaN,no,no,NaN,NaN
freq,763,974,1043,NaN,1937,1738,NaN,NaN
mean,NaN,NaN,NaN,3.666500,NaN,NaN,3.526500,34066.800000
std,NaN,NaN,NaN,2.537222,NaN,NaN,0.996892,15398.436729
min,NaN,NaN,NaN,1.000000,NaN,NaN,1.000000,12000.000000
25%,NaN,NaN,NaN,1.000000,NaN,NaN,3.000000,22800.000000
50%,NaN,NaN,NaN,3.000000,NaN,NaN,4.000000,30000.000000
75%,NaN,NaN,NaN,6.000000,NaN,NaN,4.000000,43200.000000


In [18]:
testt_df.describe(include='all')

,job_satisfaction_rate
count,2000.00000
mean,0.54878
std,0.22011
min,0.03000
25%,0.38000
50%,0.58000
75%,0.72000
max,1.00000


Правильнее, как я понимаю, писать *senior*.

In [19]:
train_df['level'] = train_df['level'].replace('sinior', 'senior')
testf_df['level'] = testf_df['level'].replace('sinior', 'senior')

Есть пропуски, заполним их в пайплайне, выбросов нет.

### Исследовательский анализ данных

Напишем функции:

In [20]:
def kdeplot_with_norm(df, features, hue):
    number_row = int(len(features)/3)
    fig, ax = plt.subplots(nrows = number_row + 1, ncols=2, figsize=(15, 5))
    ax = ax.flatten()
    for i in range(len(features)):
        sns.kdeplot(data = df, x = df[features[i]], ax=ax[i], hue=hue, common_norm=False, fill=True, alpha=0.7)
        ax[i].set_title(f'Гистограмма по столбцу {features[i]}')
        ax[i].set_xlabel('')
  

    fig.tight_layout()
    fig.show()
    

In [21]:
def countplot(df, features, hue):
    number_row = int(len(features)/3)
    fig, ax = plt.subplots(nrows = number_row + 1, ncols=2, figsize=(15, 5))
    ax = ax.flatten()
    for i in range(len(features)):
        sns.countplot(data = df, x = df[features[i]], ax=ax[i], hue=hue)
        ax[i].set_title(f'Распределение по столбцу {features[i]}')
        ax[i].set_xlabel('')
  

    fig.tight_layout()
    fig.show()

In [22]:
def boxplot(df):
    features = df.select_dtypes(include='number').columns.tolist()
    num_features = len(features)
    number_row = int(len(features)/3)
    fig, ax = plt.subplots(nrows = number_row + 1, ncols=2, figsize=(15, 10))
    ax = ax.flatten()
    for i, feature in enumerate(features):
        df.boxplot(column=features[i], ax=ax[i])
        ax[i].set_title(f'Боксплот для {feature}')

    plt.tight_layout()
    plt.show()

In [23]:
def countplot_cat(df, hue):
    features = df.select_dtypes(exclude='number').columns.tolist()
    number_row = int(len(features)/3)
    fig, ax = plt.subplots(nrows = number_row + 1, ncols=3, figsize=(15, 10))
    ax = ax.flatten()
    for i in range(len(features)):
        sns.countplot(data = df, x = df[features[i]], ax=ax[i], hue=hue)
        ax[i].set_title(f'Распределение по столбцу {features[i]}')
        ax[i].set_xlabel('')
  
    fig.delaxes(ax[-1])
    fig.delaxes(ax[-2])
    fig.delaxes(ax[-3])
    fig.tight_layout()
    fig.show()

In [24]:
def countplot_relative(df, features, hue=None):
    number_row = (len(features) // 3) + (1 if len(features) % 3 else 0)
    fig, ax = plt.subplots(nrows=number_row, ncols=3, figsize=(15, 5 * number_row))
    ax = ax.flatten()

    for i in range(len(features)):
        sns.histplot(
            data=df,
            x=df[features[i]],
            hue=hue,
            stat="percent",  
            multiple="dodge",
            shrink=0.8,
            ax=ax[i]
        )
        ax[i].set_title(f'Распределение по {features[i]} (в %)')
        ax[i].set_xlabel('')


    fig.tight_layout()
    plt.show()

Объеденим тестовые данные:

In [25]:
test_df = testf_df.join(testt_df)

Создадим категориальный столбец от `job_satisfaction_rate`:

In [26]:
sns.kdeplot(train_df, x=train_df['job_satisfaction_rate'])
plt.grid(True)
plt.title('Распределение удовлетворенности работой')
plt.xlabel('Уровень удовлетворенности')
plt.ylabel('Плотность вероятности')
plt.show()


ValueError: If using all scalar values, you must pass an index

Распределение `job_satisfaction_rate` имеет два выраженных пика – один около 0.3-0.4 и второй ближе к 0.6-0.7.

In [ ]:
train_df['job_satisfaction_rate'].plot(kind='box', figsize=(10, 8))
plt.title('Разброс показателей удовлетворенности работой')
plt.show()

In [ ]:
train_df['job_satisfaction_rate'].describe()

Возьмем медиану как границу категоризации. 

In [ ]:
train_df.loc[train_df.query('job_satisfaction_rate >= 0.56').index, 'cat_satisfaction'] = 'high'
train_df['cat_satisfaction'].fillna('low', inplace=True)

In [ ]:
test_df.loc[test_df.query('job_satisfaction_rate >= 0.56').index, 'cat_satisfaction'] = 'high'
test_df['cat_satisfaction'].fillna('low', inplace=True)

Рассмотрим числовые признаки:

*train_df*:

In [ ]:
features_num = ['salary', 'job_satisfaction_rate']
kdeplot_with_norm(train_df, features_num, hue='cat_satisfaction')

*test_df*:

In [ ]:
kdeplot_with_norm(test_df, features_num, hue='cat_satisfaction')

*train_df*:

In [ ]:
features_num2 = ['employment_years', 'supervisor_evaluation']
countplot(train_df, features_num2, hue='cat_satisfaction')

*test_df*

In [ ]:
countplot(test_df, features_num2, hue='cat_satisfaction')

*train_df*

In [ ]:
boxplot(train_df)

*test_df*

In [ ]:
boxplot(test_df)

Много недовольных работников с большим стажем. У работников с плохой оценкой недовольство выше(причинно-следственная связь здесь обратная скорее всего). Чем меньше зарплата, тем выше недовольство, но незначительно.

Рассмотрим категориальные признаки:

*train_df*

In [ ]:
countplot_cat(train_df, hue='cat_satisfaction')

*test_df*

In [ ]:
countplot_cat(test_df, hue='cat_satisfaction')

В относительных величинах больше всего недовольных в маркетинге. "Довольство" как будто не зависит от нагрузки. Большинство нарушивших трудовой договор - недовольные.

Корреляционный анализ:


*train_df*

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(train_df.corr(), annot=True, cmap="coolwarm",fmt='.2f');
plt.title("Корреляционная матрица количественных признаков по train_df")
plt.show()

*test_df*

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(test_df.corr(), annot=True, cmap="coolwarm",fmt='.2f');
plt.title("Корреляционная матрица количественных признаков по test_df")
plt.show()

Отличия корреляций *train_df* и *test_df*:

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(train_df.corr()-test_df.corr(), annot=True, cmap="coolwarm",fmt='.2f');
plt.title("Разница корреляционных матриц количественных признаков по train_df и test_df")
plt.show()

Построим корреляцию Спирмена:

*train_df*

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(train_df.corr(method='spearman'), annot=True, cmap="coolwarm",fmt='.2f', square=True);
plt.title("Корреляционная матрица количественных признаков по Спирмену по train_df")
plt.show()

*test_df*

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(test_df.corr(method='spearman'), annot=True, cmap="coolwarm",fmt='.2f', square=True);
plt.title("Корреляционная матрица количественных признаков по Спирмену по test_df")
plt.show()

Отличия корреляций по Спирмену между *train_df* и *test_df*:

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(train_df.corr(method='spearman')-test_df.corr(method='spearman'), annot=True, cmap="coolwarm",fmt='.2f');
plt.title("Разница корреляционных матриц по Спирмену по train_df и test_df")
plt.show()

In [ ]:
#Создаем новый датафрейм, потому что матрица игнорирует interval_cols
interval_df = train_df.drop(['employment_years', 'supervisor_evaluation'], axis=1)
interval_testdf = test_df.drop(['employment_years', 'supervisor_evaluation'], axis=1)
interval_cols = interval_df.select_dtypes(include='number').columns.tolist()

*train_df*

In [ ]:
phi_k_matrix = interval_df.phik_matrix(interval_cols=interval_cols)
plt.figure(figsize=(12, 10))
sns.heatmap(phi_k_matrix, annot=True, cmap="coolwarm",fmt='.2f', square=True);
plt.title("Корреляционная матрица Phi_k")
plt.show()

*test_df*

In [ ]:
phi_k_matrix = interval_testdf.phik_matrix(interval_cols=interval_cols)
plt.figure(figsize=(12, 10))
sns.heatmap(phi_k_matrix, annot=True, cmap="coolwarm",fmt='.2f', square=True);
plt.title("Корреляционная матрица Phi_k")
plt.show()

Отличия корреляций *phi_k* между *train_df* и *test_df*:

In [ ]:
phi_k_matrix = interval_df.phik_matrix(interval_cols=interval_cols) - interval_testdf.phik_matrix(interval_cols=interval_cols)
plt.figure(figsize=(12, 10))
sns.heatmap(phi_k_matrix, annot=True, cmap="coolwarm",fmt='.2f', square=True);
plt.title("Разница корреляционных матриц Phi_k между train_df и test_df")
plt.show()

Корреляционные матрицы тренировочных и тестовых данных близки. Целевой признак сильнее коррелирует со столбцом `last_year_promo` в тестовых данных.

На целевой признак влияет больше всего влияет оценка руководителя и наличие нарушений за последний год, но причинно-следственная связь здесь обратная(нарушения и низкая трудоспособность показывают, что сотрудник недоволен).

### Пайплайн

In [ ]:
#df = pd.concat([train_df, test_df], axis=0)

In [ ]:
#SMAPE
#Сумма, деленная на количество слагаемых - это среднее.
def smape(y_true, y_pred):
    return np.mean((np.abs(y_true-y_pred)/(np.abs(y_true)+np.abs(y_pred)))*2)*100

Если использовать оригинальную разбивку, то значение r2_score на тестовой выборке оказывается меньше 0 (-0.8). Поэтому конкатенируем таблицы и перемешиваем:

Найдем и удалим дубликаты после индексации признака `id`:

In [ ]:
print(train_df.duplicated().sum())
train_df = train_df.drop_duplicates()

Подготовим данные:

In [ ]:
X_train = train_df.drop(['job_satisfaction_rate','cat_satisfaction'], axis = 1)
y_train = train_df['job_satisfaction_rate']
X_test = test_df.drop(['job_satisfaction_rate','cat_satisfaction'], axis = 1)
y_test = testf_df.merge(testt_df, on='id')['job_satisfaction_rate']

In [ ]:
#Проверим, что все правильно записали
X_test.shape[1] == X_train.shape[1]

In [ ]:
#Разбиение данных на типы:
cat_col_names = X_train.select_dtypes(exclude='number').drop(['level','workload'],axis=1).columns.tolist()
ord_col_names = ['level', 'workload']
num_col_names = X_train.select_dtypes(include='number').columns.tolist()

In [ ]:
#Пайплайн:
ord_pipe = Pipeline([
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
    ('ord', OrdinalEncoder(
        categories=[['junior', 'middle', 'senior'], ['low', 'medium', 'high']],
        handle_unknown='use_encoded_value',
        unknown_value=-1
    ))
])

#Пайплайн:
ord_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ord', OrdinalEncoder(
        categories=[['junior', 'middle', 'senior'], ['low', 'medium', 'high']],
        handle_unknown='use_encoded_value',
        unknown_value=-1
    ))
])

In [ ]:
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first'))])

In [ ]:
num_pipe = Pipeline([('scaler', StandardScaler())])

data_preprocessor = ColumnTransformer([('cat', cat_pipe, cat_col_names),
                                       ('num', num_pipe, num_col_names),
                                       ('ord', ord_pipe, ord_col_names)],
                                        remainder='passthrough')

pipe_final = Pipeline([('preprocessor', data_preprocessor),
                      ('models',LinearRegression)])

param_grid = [
    {
        'models':[LinearRegression()],
        'preprocessor__num__scaler': [MinMaxScaler(), RobustScaler(), 'passthrough']
    },
    {
        'models': [DecisionTreeRegressor(random_state=1)],
        'models__max_depth': range(2, 15),
        'models__min_samples_split': range(2, 10),
        'models__min_samples_leaf': range(2, 10),
        'preprocessor__num__scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough']
    },
    ]

cv = KFold(n_splits=5, shuffle=True, random_state=1)

smape_scorer = make_scorer(smape, greater_is_better=False)

rand_search_1 = RandomizedSearchCV(
    pipe_final,
    param_grid,
    cv=cv,
    scoring=smape_scorer,
    random_state=1,
    n_jobs=-1,
    n_iter = 60
)

In [ ]:
rand_search_1.fit(X_train, y_train)

y_test_pred = rand_search_1.best_estimator_.predict(X_test)

smape_score = smape(y_test, y_test_pred)

print('Лучшая модель и её параметры:\n', rand_search_1.best_estimator_)
print(f'SMAPE на кросс-валидации: {-rand_search_1.best_score_:.4f}%')
print(f'Метрика SMAPE на тестовой выборке: {smape_score:.4f}%')

Требуемое значение *SMAPE* достигнуто. Лучшая модель: *DecisionTreeRegressor(max_depth=14, min_samples_leaf=6,
                                       min_samples_split=3, random_state=1))])*. <br> 
SMAPE на кросс-валидации: 15.2499% <br>
Метрика SMAPE на тестовой выборке: 13.8177% <br>


Используем в качестве константной модели `DummyRegressor` для проверки модели:

In [ ]:
dummy_regr = DummyRegressor(strategy="mean")

dummy_regr.fit(X_train, y_train)

DummyRegressor()

dum_pred = dummy_regr.predict(X_test)

print(f'Метрика SMAPE на dummy модели: {smape(y_test, dum_pred):.4f}%')
print(f'Метрика SMAPE на тестовой выборке: {smape_score:.4f}%')

In [ ]:
model = rand_search_1.best_estimator_.named_steps['models'] 
preprocessor = rand_search_1.best_estimator_.named_steps['preprocessor']
X_transformed = preprocessor.transform(X_test)

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_transformed)

plt.figure(figsize=(12, 10))

shap.summary_plot(
    shap_values, 
    X_transformed, 
    feature_names=preprocessor.get_feature_names_out(),
    plot_type='bar', 
    show=False
)

ax = plt.gca()
ax.set_xlabel('Средний абсолютный вклад')
ax.set_ylabel('Признаки')
ax.set_title('Влияние признаков на предсказание модели')
plt.tight_layout()
plt.show()

Наиболее значимые признаки: <br>
`supervisor_evaluation`<br>
`employment_years`<br>
`level`<br>
`salary`<br>
`cat__last_year_violations_yes`<br>

### Промежуточный вывод

Лучшая модель: DecisionTreeRegressor(max_depth=10, min_samples_leaf=2, min_samples_split=10, random_state=1). Она справилась лучше по причине того, что зависимости нелинейные, а сама модель чувстительна к выбросам. *DecisionTreeRegressor* разбивает данные на изолированные группы. Выбросы попадают в отдельные листья и не влияют на весь прогноз

## Предсказание увольнения сотрудника из компании

### Загрузка данных

In [ ]:
trainq_df=pd.read_csv('/datasets/train_quit.csv', index_col='id')
testfeat_df=pd.read_csv('/datasets/test_features.csv', index_col='id')
testtq_df=pd.read_csv('/datasets/test_target_quit.csv', index_col='id')

In [ ]:
trainq_df.sample(10)

In [ ]:
trainq_df.info()

In [ ]:
testfeat_df.sample(10)

In [ ]:
testfeat_df.info()

In [ ]:
testtq_df.sample(10)

In [ ]:
testtq_df.info()

### Предобработка данных

In [ ]:
uniques(trainq_df)

In [ ]:
uniques(testfeat_df)

In [ ]:
testfeat_df = testfeat_df.replace({' ': np.nan, 'sinior': 'senior'})
trainq_df = trainq_df.replace({'sinior': 'senior'})

In [ ]:
testfeat_df.describe(include='all')

In [ ]:
trainq_df.describe(include='all')

In [ ]:
testtq_df.describe(include='all')

In [ ]:
test2_df = testfeat_df.join(testtq_df)

### Исследовательский анализ данных

*trainq_df*

In [ ]:
features_num = ['salary']
kdeplot_with_norm(trainq_df, features_num, hue='quit')

*test2_df*

In [ ]:
features_num = ['salary']
kdeplot_with_norm(test2_df, features_num, hue='quit')

*trainq_df*

In [ ]:
features_num2 = ['employment_years', 'supervisor_evaluation']
countplot(trainq_df, features_num2, 'quit')

*test2_df*

In [ ]:
countplot(test2_df, features_num2, 'quit')

*trainq_df*

In [ ]:
boxplot(trainq_df)

*test2_df*

In [ ]:
boxplot(test2_df)

*trainq_df*

In [ ]:
countplot_cat(trainq_df, 'quit')

*test2_df*

Целевой признак несбалансирован. Тестовые и тренировочные данные похожи.

Корреляционный анализ:

*trainq_df*

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(trainq_df.corr(), annot=True, cmap="coolwarm",fmt='.2f');
plt.title("Корреляционная матрица количественных признаков по trainq_df")
plt.show()

*test2_df*

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(test2_df.corr(), annot=True, cmap="coolwarm",fmt='.2f');
plt.title("Корреляционная матрица количественных признаков по test2_df")
plt.show()

Отличия корреляций между *trainq_df* и *test_df2*:

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(trainq_df.corr() - test2_df.corr(), annot=True, cmap="coolwarm",fmt='.2f');
plt.title("Разница корреляционных матрицы количественных признаков по trainq_df и test2_df")
plt.show()

Корреляция по Спирмену:

*trainq_df*

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(trainq_df.corr(method='spearman'), annot=True, cmap="coolwarm",fmt='.2f', square=True);
plt.title("Корреляционная матрица количественных признаков по Спирмену по trainq_df")
plt.show()

*test2_df*

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(test2_df.corr(method='spearman'), annot=True, cmap="coolwarm",fmt='.2f', square=True);
plt.title("Корреляционная матрица количественных признаков по Спирмену по test2_df")
plt.show()

Отличия корреляций по Спирмену между *trainq_df* и *test_df2*:

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(trainq_df.corr(method='spearman') - test2_df.corr(method='spearman'), annot=True, cmap="coolwarm",fmt='.2f', square=True);
plt.title("Разница корреляционных матриц по Спирмену по trainq_df и test2_df")
plt.show()

*trainq_df*

In [ ]:
#Создаем новый датафрейм, потому что матрица игнорирует interval_cols
interval_df = trainq_df.drop(['employment_years', 'supervisor_evaluation'], axis=1)
interval_testdf = test2_df.drop(['employment_years', 'supervisor_evaluation'], axis=1)
interval_cols = interval_df.select_dtypes(include='number').columns.tolist()

In [ ]:
phi_k_matrix = interval_df.phik_matrix(interval_cols=interval_cols)
plt.figure(figsize=(12, 10))
sns.heatmap(phi_k_matrix, annot=True, cmap="coolwarm",fmt='.2f', square=True);
plt.title("Корреляционная матрица phi_k по trainq_df")
plt.show()

*test2_df*

In [ ]:
phi_k_matrix = interval_testdf.phik_matrix(interval_cols=interval_cols)
plt.figure(figsize=(12, 10))
sns.heatmap(phi_k_matrix, annot=True, cmap="coolwarm",fmt='.2f', square=True);
plt.title("Корреляционная матрица phi_k по test2_df")
plt.show()

Отличия корреляций *phi_k* между *trainq_df* и *test_df2*:

In [ ]:
phi_k_matrix = interval_df.phik_matrix(interval_cols=interval_cols) - interval_testdf.phik_matrix(interval_cols=interval_cols)
plt.figure(figsize=(12, 10))
sns.heatmap(phi_k_matrix, annot=True, cmap="coolwarm",fmt='.2f', square=True);
plt.title("Разница корреляционных матриц phi_k по trainq_df и test2_df")
plt.show()

Корреляционные матрицы тренировочных и тестовых данных близки. Разницы между ними несущественны.

In [ ]:
salquit = trainq_df.query('quit == "yes"')['salary'].median()
print(f'Медианная зарплата ушедшего сотрудника: {salquit}')

In [ ]:
salnoquit = trainq_df.query('quit == "no"')['salary'].median()
print(f'Медианная зарплата оставшегося сотрудника: {salnoquit}')

Составим портрет уволившегося сотрудника:

Построим релевантные графики:

In [ ]:
trainq_df

In [ ]:
cat_cols = ['level', 'dept', 'workload', 'last_year_promo', 'last_year_violations', 'employment_years', 'supervisor_evaluation']

for col in cat_cols:
    level_total = trainq_df[col].value_counts(normalize=True)
    level_quit = trainq_df.query('quit=="yes"')[col].value_counts(normalize=True)
    
    compare_df = pd.DataFrame({
        'Все сотрудники': level_total * 100,
        'Уволившиеся': level_quit * 100
    }).sort_index()
    
    plt.figure(figsize=(15, 12))
    ax = compare_df.plot.bar()
    
    plt.title(f'Распределения должностей среди всех сотрудников и уволившихся "{col}"')
    plt.xlabel('Категории признака')
    plt.ylabel('Доля, %')
    plt.grid(axis='y')
    plt.legend(bbox_to_anchor=(1.05, 1))
    
    plt.tight_layout()
    plt.show()

Работает 1 год; <br> 
Медианная зарплата: 24000.0;
Имеет оценку руководителя "3"; <br>
Должность *junior*; <br>
Уровень нагрузки средний или низкий; <br>
Из отдела продаж (в абсолютном значении оттуда уволилось больше всего); <br>
Очевидно, не получал повышения в прошлом году и не нарушал трудовой договор.

Проверка связи `job_satisfaction_rate` и `quit`:

In [ ]:
#Создаем датафрейм с целевыми признаками:
test_quit_job = test2_df.join(testt_df, on='id')

In [ ]:
sns.histplot(data=test_quit_job, x='job_satisfaction_rate', hue='quit', 
             stat='density', common_norm=False, 
             bins=35, palette={'no':'blue', 'yes':'red'})

plt.title('Распределение удовлетворенности работой для ушедших и оставшихся')
plt.xlabel('Удовлетворенность работой')
plt.ylabel('Плотность распределения')
plt.legend(title='Уволился', labels=['Да', 'Нет'])
plt.show()

`quit` явно разделен на две области.

Проведем статитеский анализ:

H0: Уровень удовлетворенности не влияет на вероятность уволиться.

H1: Сотрудники с низкой удовлетворенностью чаще увольняются.

In [ ]:
alpha = 0.05

#Односторонний ttest
result = stats.ttest_ind(
    test_quit_job[test_quit_job['quit'] == 'yes']['job_satisfaction_rate'],
    test_quit_job[test_quit_job['quit'] == 'no']['job_satisfaction_rate'],
    alternative = 'less'
)
print(f"t-тест: p-value = {result.pvalue}")

if result.pvalue < alpha:
    print("Отвергаем нулевую гипотезу. Сотрудники с низкой удовлетворенностью чаще увольняются.")
else:
    print("Не отвергаем нулевую гипотезу.")

Добавим `job_satisfaction_rate` к новым таблицам:

In [ ]:
trainq_df['job_satisfaction_rate'] = rand_search_1.predict(trainq_df.drop('quit', axis=1))

test2_df['job_satisfaction_rate'] = rand_search_1.predict(test2_df.drop('quit', axis=1))

Построим *phi_k*:

In [ ]:
interval_cols = trainq_df.select_dtypes(include='number').columns.tolist()
phi_k_matrix = trainq_df.phik_matrix(interval_cols=interval_cols)
plt.figure(figsize=(12, 10))
sns.heatmap(phi_k_matrix, annot=True, cmap="coolwarm",fmt='.2f', square=True);
plt.title("Корреляционная матрица phi_k")
plt.show()

Все признаки влияют на целевой кроме признака `dept`. Больше всего на увольнение влияют: стаж, зарплата, уровень удовлетворенности работой.

### Пайплайн

Найдем и удалим дубликаты после индексации признака `id`:

In [ ]:
print(trainq_df.duplicated().sum())
trainq_df = trainq_df.drop_duplicates()

Подготовим данные:

In [ ]:
X_train = trainq_df.drop(['quit'], axis = 1)
X_test = test2_df.drop(['quit'], axis = 1)
le = LabelEncoder()
y_train = le.fit_transform(trainq_df['quit'])
y_test = le.transform(test2_df['quit'])
print(f'Кодирование классов: {le.classes_}')
print(f'y_train unique: {np.unique(y_train)}')

In [ ]:
#Разбиение данных на типы:
cat_col_names = X_train.select_dtypes(exclude='number').drop(['level', 'workload'],axis=1).columns.tolist()
ord_col_names = ['level', 'workload']
num_col_names = trainq_df.select_dtypes(include='number').columns.tolist()

In [ ]:
#Пайплайн:
ord_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ord', OrdinalEncoder(categories=[['junior', 'middle', 'senior'], ['low', 'medium', 'high']]))
])

cat_pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore'))])

num_pipe = Pipeline([('scaler', StandardScaler())])

data_preprocessor = ColumnTransformer([('cat', cat_pipe, cat_col_names),
                                       ('num', num_pipe, num_col_names),
                                       ('ord', ord_pipe, ord_col_names)])


pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', LogisticRegression(
        random_state=1,
        class_weight='balanced'
    ))
])

param_grid = [
    {
        'models':[LogisticRegression(random_state=1,
                                     solver='liblinear',
                                     penalty='l1',
                                     class_weight='balanced')],
        'preprocessor__num__scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough']
    },
    {
        'models':[KNeighborsClassifier()],
        'models__n_neighbors': range(7,15),
        'models__p': [1, 2],
        'preprocessor__num__scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough']
    },
    {
        'models': [DecisionTreeClassifier(random_state=1, class_weight='balanced')],
        'models__max_depth': range(2,5),
        'models__max_features': range(2,5),
        'models__min_samples_split': range(2,5),
        'preprocessor__num__scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough'],
        'models__min_samples_leaf': range(2,5),
        'models__ccp_alpha': [0.0, 0.01, 0.1]
    }]

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

rand_search = RandomizedSearchCV(
    pipe_final,
    param_grid,
    cv=cv,
    scoring='roc_auc',
    random_state=1,
    n_jobs=-1,
    n_iter = 35
)

rand_search.fit(X_train, y_train)

y_test_pred = rand_search.predict(X_test)
y_test_pred_proba = rand_search.predict_proba(X_test)[:, 1]  

print('Лучшая модель и её параметры:\n\n', rand_search.best_estimator_)
print(f'Средняя ROC-AUC на кросс-валидации: {rand_search.best_score_:.4f}')

In [ ]:
print(f'Метрика ROC-AUC на тестовой выборке: {roc_auc_score(y_test, y_test_pred_proba):.4f}')
print(f'Метрика f1_score на тестовой выборке: {f1_score(y_test, y_test_pred):.4f}')
print(f'Метрика recall_score на тестовой выборке: {recall_score(y_test, y_test_pred):.4f}')

Лучшая модель: *KNeighborsClassifier* <br>
Средняя *ROC-AUC* на кросс-валидации: 0.8978 <br>
Метрика *ROC-AUC* на тестовой выборке: 0.9119 <br>
Метрика *f1_score* на тестовой выборке: 0.7871 <br>
Метрика *recall_score* на тестовой выборке: 0.7376 <br>

Метрику *recall_score* используем потому что нам важно выявить увольняющегося сотрудника. Дешевле сохранить работника, чем искать ему замену.

In [ ]:
dummy_regr = DummyClassifier(strategy="stratified")

dummy_regr.fit(X_train, y_train)

DummyRegressor()

dum_pred = dummy_regr.predict(X_test)

print(f'Метрика ROC-AUC на dummy модели: {roc_auc_score(y_test, dum_pred):.4f}')
print(f'Метрика ROC-AUC на тестовой выборке: {roc_auc_score(y_test, y_test_pred_proba):.4f}')

Модель адекватна.

Построим график убывания средних *SHAP*‑значений:

**В прошлом проекте получилось построить этот график, сейчас получается так, как я тут оставил.**

In [ ]:
X_transformed = rand_search_1.best_estimator_.named_steps['preprocessor'].transform(X_train)

explainer = shap.Explainer(rand_search_1.best_estimator_.named_steps['models'], X_transformed)

shap_values = explainer.shap_values(X_transformed)

plt.figure(figsize=(12, 10))

shap.summary_plot(
    shap_values, 
    X_transformed, 
    feature_names=rand_search_1.best_estimator_.named_steps['preprocessor'].get_feature_names_out(), 
    show=False
)


ax = plt.gca()
ax.set_xlabel("SHAP значение")
ax.set_ylabel("Признаки")
ax.set_title('SHAP value признаков в порядке убывания значимости')
plt.tight_layout()
plt.show()

Наиболее важные признаки:

1. `supervisor_evaluation` 
2. `employment_years` 
3. `level`  
4. `cat_last_year_violiations_yes`
5. `salary`

# Выводы

1. В результате предобработки данных в обеих задачах заменены неправильные названия значений, пропуски вида " " заменены на пропуски вида *NaN*. Выбросов нет, пропуски будут заполнены в пайплайне. Удалены дубликаты после индексации по `id` <br><br>
2. Портерт уволившегося сотрудника:
   1.  Работает 1 год; <br> 
   2. Медианная зарплата: 24000.0; <br>
   3. Имеет оценку руководителя "3"; <br>
   4. Должность *junior*; <br>
   5. Уровень нагрузки средний или низкий; <br>
   6. Из отдела продаж (в абсолютном значении оттуда уволилось больше всего); <br>
   7. Не получал повышения в прошлом году и не нарушал трудовой договор. <br>

    Все признаки влияют на шанс уйти с работы признака `dept`. Больше всего на увольнение влияют: стаж, зарплата, уровень удовлетворенности работой.

    На уровень удовлетворенности работой больше всего влияет оценка руководителя и наличие нарушений за последний год, но причинно-следственная связь здесь обратная(нарушения и низкая трудоспособность показывают, что сотрудник недоволен). <br><br>

3. Обучение модели на целевом признаке `job_satisfaction_rate`: 

    Требуемое значение *SMAPE* достигнуто. Лучшая модель: *DecisionTreeRegressor(max_depth=14, min_samples_leaf=6,
                                       min_samples_split=3, random_state=1))])*. <br> 
    SMAPE на кросс-валидации: 15.2499% <br>
    Метрика SMAPE на тестовой выборке: 13.8177% <br>
   Критерий успеха достигнут.<br>
   Модель справилась лучше по причине того, что зависимости нелинейные, а сама модель чувстительна к выбросам. *DecisionTreeRegressor* разбивает данные на изолированные группы. Выбросы попадают в отдельные листья и не влияют на весь прогноз.<br>
   Наиболее значимые признаки при оценке уровня удовлетворенности: <br>
    1. `supervisor_evaluation`<br>
    2. `employment_years`<br>
    3. `level`<br>
    4. `salary`<br>
    5. `cat__last_year_violations_no`<br>
   
   Обучение модели на целевом признаке `quit`:
   
    Лучшая модель: *KNeighborsClassifier* <br>
    Средняя *ROC-AUC* на кросс-валидации: 0.8978 <br>
    Метрика *ROC-AUC* на тестовой выборке: 0.9119 <br>
    Метрика *f1_score* на тестовой выборке: 0.7871 <br>
    Метрика *recall_score* на тестовой выборке: 0.7376 <br>
    Критерий успеха достигнут. <br>
    Метрику *recall_score* используем потому что нам важно выявить увольняющегося сотрудника. Дешевле сохранить работника, чем искать ему замену.
    Наиболее значимые признаки при оценке вероятности увольнения: <br>
    1. `supervisor_evaluation` 
    2. `employment_years` 
    3. `level`  
    4. `cat_last_year_violiations_yes`
    5. `salary`
4. Рекомендации для бизнеса по снижению уровня увольнений:
    1. "Работать" с сотрудниками с низкой оценкой от руководителя
    1. Вести контроль за удовлетворенностью сотрудников
    2. Искать способы помогать "переживать" первый год работы
    3. Учитывать условия конкретной сферы бизнеса при использовании предложенных моделей. (я проработал 3 года стажером в НИИ, потому что это было выгоднее, чем инженером)